# Clustering Crypto

In [55]:
# Initial imports
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import OrdinalEncoder

### Fetching Cryptocurrency Data

In [56]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [57]:
# Create a DataFrame
response = requests.get(url)
response_json = response.json()

In [58]:
crypto_df = pd.DataFrame(response_json['Data']).transpose()
crypto_df.info()
crypto_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 8393 entries, 42 to OLAND
Data columns (total 36 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Id                    8393 non-null   object
 1   Url                   8393 non-null   object
 2   ImageUrl              8390 non-null   object
 3   ContentCreatedOn      8393 non-null   object
 4   Name                  8393 non-null   object
 5   Symbol                8393 non-null   object
 6   CoinName              8393 non-null   object
 7   FullName              8393 non-null   object
 8   Description           8393 non-null   object
 9   AssetTokenStatus      8393 non-null   object
 10  Algorithm             8393 non-null   object
 11  ProofType             8393 non-null   object
 12  SortOrder             8393 non-null   object
 13  Sponsored             8393 non-null   object
 14  Taxonomy              8393 non-null   object
 15  Rating                8393 non-null   obj

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.000244,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [70]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply'
dataset = crypto_df.drop(crypto_df.columns.difference([
    'CoinName', 
    'Algorithm', 
    'IsTrading', 
    'ProofType', 
    'TotalCoinsMined', 
    'MaxSupply',
]), axis='columns')

dataset.tail()

,CoinName,Algorithm,ProofType,IsTrading,TotalCoinsMined,MaxSupply
VTS,Veritise,N/A,N/A,False,0,300000000
ARTEM,Artem,N/A,N/A,True,1000000000,-1
SKYRIM,Skyrim Finance,N/A,N/A,True,100000000,-1
WOLFILAND,Wolfiland,N/A,N/A,False,1653250,-1
OLAND,Oceanland,N/A,N/A,False,960000000,-1


In [71]:
# Keep only cryptocurrencies that are trading
dataset = dataset.loc[dataset['IsTrading'] == True]

# Keep only cryptocurrencies with a working algorithm
dataset = dataset.loc[dataset['Algorithm'] != 'N/A']

# Remove the "IsTrading" column
dataset.drop(columns='IsTrading', inplace=True)

# Remove rows with cryptocurrencies having no coins mined
dataset = dataset.loc[dataset['TotalCoinsMined'] > 0]

# Drop rows where 'N/A' text values exist
dataset.replace('N/A', np.nan, inplace=True)
dataset.dropna(inplace=True)

# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = dataset['CoinName'].to_frame()

# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
dataset.drop(columns='CoinName', inplace=True)

dataset.tail()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
ETH,Ethash,PoW,1.209980e+08,-1.0
NAV,X13,PoW/PoS,7.359654e+07,-1.0
NXS,SHA3,PoW/nPoS,7.351290e+07,78000000.0
NVC,Scrypt,PoW/PoS,3.718881e+06,-1.0
POA,Proof-of-Authority,PoA,1.608537e+08,-1.0


In [72]:
# Create dummy variables for text features
X = pd.get_dummies(dataset)
X.head()

,TotalCoinsMined,MaxSupply,Algorithm_Autolykos,Algorithm_BEP-2,Algorithm_BEP-20 Token,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake2B + SHA3,Algorithm_Blake2b,Algorithm_C31,...,ProofType_PoW/PoSe,ProofType_PoW/nPoS,ProofType_ProgPoW/PoS,ProofType_Proof of Authority,ProofType_Proof-of-Work,ProofType_SPoS,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW,ProofType_dPoW/PoW
42,4.199995e+01,42.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
NSR,6.178721e+09,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TRI,1.991732e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CMTC,8.728300e+05,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CHAT,1.000000e+09,-1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)
X_scaled.shape

(140, 83)

### Reducing Dimensions Using PCA

In [74]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [75]:
# Create a DataFrame with the principal components data
pc_df = pd.DataFrame(
    data=pca.fit_transform(X_scaled), 
    columns=['PC1', 'PC2', 'PC3'], 
    index=dataset.index,
)

pc_df.head()

,PC1,PC2,PC3
42,0.222849,-1.326062,-1.343658
NSR,0.696262,-1.169300,-0.312068
TRI,0.654792,-1.974566,-1.626910
CMTC,-0.853523,0.447471,-0.364626
CHAT,0.222854,-1.326058,-1.343658


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [76]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pc_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Create the Elbow Curve using hvPlot
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [108]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=42)

# Fit the model
model.fit(pc_df)

# Predict clusters
predictions = model.predict(pc_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat(
    [coin_names, dataset, pc_df],
    join='inner', 
    axis='columns',
)

clustered_df['Class'] = predictions

clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply,PC1,PC2,PC3,Class
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42.0,0.222849,-1.326062,-1.343658,0
NSR,NuShares,PoS,PoS,6.178721e+09,0.0,0.696262,-1.169300,-0.312068,0
TRI,Triangles Coin,X13,PoW/PoS,1.991732e+05,0.0,0.654792,-1.974566,-1.626910,0
CMTC,CometCoin,Scrypt,PoW,8.728300e+05,0.0,-0.853523,0.447471,-0.364626,2
CHAT,OpenChat,Scrypt,PoW/PoS,1.000000e+09,-1.0,0.222854,-1.326058,-1.343658,0


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [109]:
# Scale data to create the scatter plot
scatter_data = MinMaxScaler().fit_transform(
    clustered_df[['TotalCoinsMined', 'MaxSupply']]
)

scatterplot_df = pd.DataFrame(
    data=scatter_data, 
    columns=['TotalCoinsMined', 'MaxSupply']
)

scatterplot_df.insert(0, 'CoinName', coin_names.values)
scatterplot_df['Class'] = clustered_df['Class'].values

scatterplot_df.head()

,CoinName,TotalCoinsMined,MaxSupply,Class
0,42 Coin,0.000000e+00,2.047619e-12,0
1,NuShares,6.241132e-06,4.761905e-14,0
2,Triangles Coin,2.011427e-10,4.761905e-14,0
3,CometCoin,8.816040e-10,4.761905e-14,2
4,OpenChat,1.010101e-06,0.000000e+00,0


In [110]:
# Plot the PCA clusters
clustered_df.hvplot.scatter(
    x='PC1', 
    y='PC2', 
    by='Class', 
    hover_cols=['CoinName', 'Algorithm', 'MaxSupply', 'TotalCoinsMined'],
    title='fuck'
)

:NdOverlay   [Class]
   :Scatter   [PC1]   (PC2,CoinName,Algorithm,MaxSupply,TotalCoinsMined)

In [111]:
# Plot the scatter with x="TotalCoinsMined" and y="MaxSupply"
scatterplot_df.hvplot.scatter(
    x='TotalCoinsMined', 
    y='MaxSupply', 
    by='Class', 
    hover_cols='CoinName'
)

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (MaxSupply,CoinName)

#### Table of Tradable Cryptocurrencies

In [114]:
# Table with tradable cryptos
clustered_df.sort_values(by='CoinName').hvplot.table(
    columns=[
        'CoinName', 
        'Algorithm', 
        'ProofType', 
        'MaxSupply', 
        'TotalCoinsMined', 
        'Class',
    ], 
    sortable=True, 
    selectable=True)

:Table   [CoinName,Algorithm,ProofType,MaxSupply,TotalCoinsMined,Class]

In [115]:
# Print the total number of tradable cryptocurrencies
print(f"There are {clustered_df.shape[0]} tradable cryptocurrencies.")

There are 140 tradable cryptocurrencies.
